Solution for the problem statement.

In [ ]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from skimage.util.montage import montage2d as montage
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
test_image_dir = '../input/datadfy'
from keras.models import load_model #to load pretrained model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input

In [ ]:
IMG_SIZE = (299, 299) # [(224, 224), (384, 384), (512, 512), (640, 640)]
BATCH_SIZE = 1
RGB_FLIP = 1 # should rgb be flipped when rendering images

In [ ]:
masks = pd.read_csv('../input/datadfy/mysubmission.csv')
print(masks.shape[0], 'masks found')
print(masks['ImageId'].value_counts().shape[0])
masks.head()

In [ ]:
valid_args = dict(fill_mode = 'reflect',
                   data_format = 'channels_last',
                  preprocessing_function = preprocess_input)

valid_idg = ImageDataGenerator(**valid_args)

In [ ]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

Loading pretrained model


In [ ]:
model = load_model('../input/transfer-learning-for-boat-or-no-boat/full_ship_model.h5')

In [ ]:
model.summary()


#We use the mysubmission file as the basis for loading and running the image.

In [ ]:
submission_df = pd.read_csv('../input/datadfy/mysubmission.csv')
submission_df['path'] = submission_df['ImageId'].map(lambda x: os.path.join(test_image_dir, x))
submission_df

We will use the same generator as before to read and preprocess images

In [ ]:
test_gen = flow_from_dataframe(valid_idg, 
                               submission_df, 
                             path_col = 'path',
                            y_col = 'ImageId', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = BATCH_SIZE, 
                              shuffle = False)

In [ ]:
fig, m_axs = plt.subplots(1, 2, figsize = (20, 30))
ax1 = m_axs[0]
ax2 = m_axs[1]
t_x = test_gen[0][0]
t_y = test_gen[0][1]
print(t_y)
t_y = model.predict(t_x)
print("Prediction Score is : ",t_y)
t_stack = ((t_x-t_x.min())/(t_x.max()-t_x.min()))[:, :, :, ::RGB_FLIP]
ax1.imshow(montage_rgb(t_stack))
ax1.set_title('images')
alpha_stack = np.tile(np.expand_dims(np.expand_dims(t_y, -1), -1), [1, t_stack.shape[1], t_stack.shape[2], 1])
rgba_stack = np.concatenate([t_stack, alpha_stack], -1)
ax2.imshow(montage_rgb(rgba_stack))
ax2.set_title('boats')